In [1]:
!pip install datasets torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu124 transformers[torch] accelerate -U


zsh:1: no matches found: transformers[torch]


In [2]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
import numpy as np
import torch


/opt/anaconda3/envs/devenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Cargamos el conjunto de datos imdb valoraciones

In [3]:
dataset = load_dataset("imdb")

In [4]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def tokenizador(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

data_tokenizada = dataset.map(tokenizador, batched=True)

data_tokenizada = data_tokenizada.remove_columns(["text"])
data_tokenizada = data_tokenizada.rename_column("label", "labels")
data_tokenizada.set_format("torch")

Map: 100%|██████████| 50000/50000 [00:06<00:00, 7693.06 examples/s]


In [5]:
train_dataset = data_tokenizada["train"]
test_dataset = data_tokenizada["test"]


In [6]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    report_to="none"
)


In [8]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


/var/folders/lp/4ygb41997bs8wflrf0qz0zf00000gn/T/ipykernel_85868/754301748.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/opt/anaconda3/envs/devenv/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [9]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [10]:
#print(torch.cuda.is_available())
#if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Usando GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Usando CPU")

IndentationError: unexpected indent (2417680616.py, line 3)

In [ ]:
!nvcc --version

In [11]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


  1%|          | 9/1563 [00:10<30:27,  1.18s/it]

KeyboardInterrupt: 

In [ ]:
results = trainer.evaluate()
print(results)


In [ ]:
model.save_pretrained("./distilbert-finetuned-imdb")
tokenizer.save_pretrained("./distilbert-finetuned-imdb")

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

model = DistilBertForSequenceClassification.from_pretrained("./distilbert-finetuned-imdb")
tokenizer = DistilBertTokenizerFast.from_pretrained("./distilbert-finetuned-imdb")